# Cuaderno de requerimento para contagio y reservas

In [ ]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\codigos"))
from utils import calcular_cuota, ingresos_financieros, ingreso_fin_entre_fechas
import warnings
warnings.filterwarnings("ignore")
import numpy as np

In [ ]:
base_contagio = pd.read_csv("../db/base_contagio.csv", sep=";")
cols_activos = ["DPI", "nomentidad", "nomvinculo", "desctipoactivo", 
                "fecconce", "fecvence", "capitalorig"]

base_contagio.drop_duplicates(subset=cols_activos, inplace=True)
# base_contagio = base_contagio[base_contagio["CategoriaContagio"].isin(["C", "D", "E"])]
# Ver precalificaciones de Horus
horus_compra_saldo = pd.read_csv("../db/compra_saldo.csv")
cols_compra_sldo = ["DxDpi", "DxTipoCliente", "DfSalarioBase", "DfCoutaTotalCreditosExternosConsolidados", 
 "DfRciGlobalInicialDeudor", "DfRciInternoInicialDeudor", "DfIngresosValidados", 
 "DfMontoMaximoOfertado"]
horus_compra_saldo = horus_compra_saldo[cols_compra_sldo]


In [ ]:
base_contagio_temp = base_contagio.groupby(by="DPI")["saldo"].sum().reset_index()
base_contagio_temp.columns = ["DPI", "SALDO EXTERNO CLIENTE"]
base_contagio = pd.merge(base_contagio, base_contagio_temp, how="inner", on="DPI")

In [ ]:
base_contagio["CategoriaContagio"].value_counts()

In [ ]:
base_con_precas = pd.merge(base_contagio, horus_compra_saldo, how="left", 
                           left_on="DPI", right_on="DxDpi")

In [ ]:
# Corregir el error en los vacíos
base_con_precas["ESTADO_COLOR"] = np.where(base_con_precas["DxDpi"].isna(), 0, 1)
base_con_precas["PAGA RESERVAS"] = np.where(base_con_precas["CANT_CRED_NO_CUMPLE"] == 0, 0, 1)
conteo_nivel_cliente = base_con_precas.drop_duplicates(subset="DPI")
conteo_nivel_cliente["ESTADO_COLOR"].value_counts().reset_index(name="CLIENTES")

## P1

In [ ]:
mascar_ped = base_con_precas["SEGMENTO"].isin(["ESTADO Y DESCENTRALIZADAS", "PREFERENTES"])
mascar_cons_interna = (base_con_precas["PAGA RESERVAS"] == 0) & (base_con_precas["PAGA RESERVAS"] == 0) & (base_con_precas["DfRciGlobalInicialDeudor"]< 50) & (base_con_precas["DfRciInternoInicialDeudor"] < 60)
mascar_cons_ext = (~base_con_precas["DfRciGlobalInicialDeudor"].isna())

In [ ]:
ped = base_con_precas.loc[mascar_ped]

# Contar para crear matriz de consolidación
a11 = len(ped.loc[mascar_cons_interna & mascar_cons_ext])
a21 = len(ped.loc[mascar_cons_ext & ~mascar_cons_interna])
conteos_si_externa = [a11, a21]

a12 = len(ped.loc[mascar_cons_interna & ~mascar_cons_ext])
a22 = len(ped.loc[~mascar_cons_interna & ~mascar_cons_ext])
conteos_no_externa = [a12, a22]

dic_matriz = {"Sí": conteos_si_externa, "No": conteos_no_externa}
df_matriz = pd.DataFrame.from_dict(dic_matriz)
df_matriz.to_latex("../db/Matriz_consolidaciones.txt")

Ver a quienes se les puede comprar el saldo contagiado porque sí pasan política y ver quiénes no. Evaluar a quiénes sí se les puede hacer recrédito. 

Se establece a alguien como posible cliente para comprar su deuda interna si cumple le pago de 15% a capital y su RCI permite pasar política.

## Sí pasa política

In [ ]:
pasa_politica = ped["ESTADO_COLOR"] == 0
cols_resumen = ["TOTAL_DESEMBOLSO", "TOTAL_SALDO", "saldo", "capitalorig"]
for col in cols_resumen:
    print(f"{col} para Remesa es: {ped.loc[pasa_politica][col].sum() / 10**6} M")

In [ ]:
import matplotlib.pyplot as plt

# Graficar relaciones de interés
plt.figure(figsize=(8,6))
plt.pie(
    ped["PAGA RESERVAS"].value_counts(),
    labels=["No", "Sí"],
    colors=["#dc0464", "#22aeae"],
    autopct='%1.1f%%'  # Agrega los porcentajes con un decimal
)
plt.title("¿Cliente cumple 15% de pago a capital?")
plt.show()


In [ ]:
# Filtrar los datos y contar según orden alfabético
conteo_alfabetico_interna = ped.drop_duplicates(subset="DPI")["CATEGORIA_INTERNA"].value_counts().sort_index()

# Graficar los datos en orden alfabético
plt.figure(figsize=(8,6))
conteo_alfabetico_interna.plot(kind="bar", color=["#dc0464", "#22aeae", "#a3abb3", "#fbdc05", "#f2ebea"])

# Etiquetas y título
plt.xlabel("Categoría interna")
plt.ylabel("Clientes")
plt.title("Clientes según categoría interna")
plt.show()

In [ ]:
# Filtrar los datos y contar según orden alfabético
conteo_alfabetico = ped.drop_duplicates(subset="DPI")["CategoriaContagio"].value_counts().sort_index()

# Graficar los datos en orden alfabético
plt.figure(figsize=(8,6))
conteo_alfabetico.plot(kind="bar", color=["#dc0464", "#22aeae", "#a3abb3", "#fbdc05", "#f2ebea"])

# Etiquetas y título
plt.xlabel("Categoría externa")
plt.ylabel("Clientes")
plt.title("Clientes según categoría externa")
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
ped.drop_duplicates(subset="DPI")["CategoriaContagio"].value_counts().plot(kind="bar", color=["#dc0464", "#22aeae", 
                                                                   "#a3abb3", "#fbdc05",
                                                                   "#f2ebea"])
plt.xlabel("Categoría externa")
plt.ylabel("Clientes")
plt.title("Clientes según categoría externa")

## Buscar datos faltantes en Zynapse

In [ ]:
# with open ("../db/dpis.txt", "w") as f:
#     for dpi in ped["DPI"].unique().tolist():
#         f.write(f"'{dpi}',")

In [ ]:
# no_precas = pd.read_csv("../db/precas_dpis.csv")
# ped = pd.merge(ped, no_precas, how="left", left_on="DPI", right_on="DxDpi")

In [ ]:
# with open ("../db/precas.txt", "w") as f:
#     for preca in ped["KnNoPrecalificacion"].unique().tolist():
#         try:
#             f.write(f"'{int(preca)}',")
#         except:
#             pass

In [ ]:
# datos_demograficos = pd.read_csv("../db/datos_demograficos.csv")
# ped = pd.merge(ped, datos_demograficos, how="left", left_on="KnNoPrecalificacion", right_on="no_precalificacion")

In [ ]:
plt.figure(figsize=(8,6))
plt.hist(ped["DfIngresosValidados"], bins=150, color="#22aeae")
plt.xlabel("Ingresos Validados")
plt.xlim([1000, 40000])
plt.ylim([0,1300])
plt.ylabel("Clientes")
plt.title("Histograma de ingresos validados")
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.hist(ped["DfRciGlobalInicialDeudor"], bins=100, color="#dc0464")
plt.xlabel("RCI Global Inicial")
plt.xlim([0, 100])
plt.ylim([0,200])
plt.ylabel("Clientes")
plt.title("Histograma de RCI Global inicial")
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.hist(ped["DfRciInternoInicialDeudor"], bins=75, color="#dc0464")
plt.xlabel("RCI Interno Inicial")
plt.xlim([0, 70])
plt.ylim([0,300])
plt.ylabel("Clientes")
plt.title("Histograma de RCI Interno inicial")
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.hist(ped["DfCoutaTotalCreditosExternosConsolidados"], bins=75, color="#22aeae")
plt.xlabel("Suma de cuotas externas")
plt.xlim([0, 10000])
plt.ylim([0,500])
plt.ylabel("Clientes")
plt.title("Histograma de suma de cuota de créditos externos")
plt.show()

## Estimaciones de montos para contagiados

In [ ]:
contagiados = ped[ped["CategoriaContagio"].isin(["C", "D", "E"]) & (ped["CATEGORIA_INTERNA"].isin(["A", "B", "C", "D"]))]
dic_reservas = {"C": 0.45*0.156, "D": 0.325*0.45, "E": 1}
contagiados["MAXIMA CATEGORIA"] = contagiados.apply(lambda x: max(x["CategoriaContagio"], x["CATEGORIA_INTERNA"]), axis=1)
contagiados["PORCENTAJE RESERVAS"] = contagiados.apply(lambda x: dic_reservas.get(x["MAXIMA CATEGORIA"]), axis=1)
contagiados["PORCENTAJE RESERVAS ACTUAL"] = contagiados.apply(lambda x: dic_reservas.get(x["CATEGORIA_INTERNA"]), axis=1)


contagiados["RESERVAS A PAGAR CONTAGIO"] = contagiados["PORCENTAJE RESERVAS"] * contagiados["TOTAL_SALDO"]
contagiados["RESERVAS ACTUALES"] = contagiados["PORCENTAJE RESERVAS ACTUAL"] * contagiados["TOTAL_SALDO"]

contagiados["SE CONTAGIA"] = np.where(contagiados["CATEGORIA_INTERNA"] != contagiados["MAXIMA CATEGORIA"], 1, 0)

In [ ]:
contagio_nivel_cliente = contagiados.drop_duplicates(subset="DPI")
contagio_nivel_cliente.agg(
    RESERVAS_CONTAGIO=("RESERVAS A PAGAR CONTAGIO", "sum"),
    SALDO_OTORGADO=("TOTAL_DESEMBOLSO", "sum"),
)

reservas_contagio = contagio_nivel_cliente.loc[contagio_nivel_cliente["SE CONTAGIA"] == 1]["RESERVAS A PAGAR CONTAGIO"].sum()
reservas_obligatorias = contagio_nivel_cliente.loc[contagio_nivel_cliente["SE CONTAGIA"] == 1]["RESERVAS ACTUALES"].sum()
print(f"Las reservas por contagio serían de {(reservas_contagio - reservas_obligatorias) / 10**6} M")

In [ ]:
contagio_nivel_cliente["MAXIMA CATEGORIA"].value_counts()

In [ ]:
import seaborn as sns

conteo_cruzado = pd.crosstab(contagio_nivel_cliente["CATEGORIA_INTERNA"], contagio_nivel_cliente["CategoriaContagio"])
plt.figure(figsize=(8,6))
sns.heatmap(conteo_cruzado, annot=True, fmt="d", cmap="YlGnBu", cbar=True)
plt.xlabel("Categoría externa")
plt.ylabel("Categoría interna")
plt.title("Distribución de categorías internas y externas")
plt.show()

## Ver razones de rechazo

In [ ]:
rechazo = pd.read_csv("../db/razones_rechazo.csv")

In [ ]:
dpis = contagiados["DPI"].unique().tolist()
rechazo["CONTAGIADO"] = np.where(rechazo["dpi"].isin(dpis), 1, 0)
rechazo = rechazo[rechazo["CONTAGIADO"] == 1]

In [ ]:
# Ahora veamos cuáles son los motivos de rechazo
cols = ["dpi", "no_precalificacion", "SEGMENTO_PRODUCTO",
        'rciGlobalInicialDeudor', 'rciGlobalFinalDeudor',
       'rciInternoFinalDeudor', 'rciInternoInicialDeudor',
        "nombrepatrono", 'monto', 'plazo', 'tasa',
        'conSaldoCreditoBT', 'conSaldoCreditoExt', 'conSaldoFactorajeExt',
        'conSaldoTcBT', 'conSaldoTcExt', 'tipoCliente',
        "scoreDeudorInstallmentQnt",
        "scoreDeudorInstallmentQualPacing",
        "reglaNombre", "tipooferta", "Oferta"]

rojas = rechazo[cols]
rojas = pd.get_dummies(rojas, columns=["reglaNombre"])

cols_one_hot = [col for col in rojas.columns if "reglaNombre_" in col]
rojas = rojas.groupby(by="dpi").agg({**{col: 'max' for col in cols_one_hot}, 
                                     **{col: "first" for col in rojas.columns.tolist() if col not in cols_one_hot
                                        and col != "dpi"}}).reset_index()

for col in cols_one_hot:
    rojas[col] = rojas[col].astype(str).str.replace("True", "1")
    rojas[col] = rojas[col].astype(str).str.replace("False", "0")
    rojas[col] = rojas[col].astype("int64")

    

In [ ]:
conteo = {}
for col in cols_one_hot:
    conteo[col] = [rojas[col].sum()]

df_conteo = pd.DataFrame.from_dict(conteo)
df_conteo

In [ ]:
principales = []
for col in df_conteo.columns:
    if df_conteo.loc[0, col] >= 115:
        principales.append(col)

df_principales = df_conteo[principales]

x = np.arange(len(df_principales))
colores = colores = [
    '#FF5733',  # Rojo anaranjado
    '#33FF57',  # Verde
    '#3357FF',  # Azul
    '#F1C40F',  # Amarillo
    '#9B59B6',  # Púrpura
    '#1ABC9C',  # Turquesa
    '#E74C3C',  # Rojo intenso
    '#3498DB',  # Azul claro
    '#2ECC71',  # Verde claro
    '#E67E22',  # Naranja
    '#ECF0F1',  # Gris claro
    '#95A5A6',  # Gris oscuro
    '#D35400',  # Naranja oscuro
    '#16A085',  # Verde azulado
    '#8E44AD',  # Púrpura oscuro
    '#27AE60',  # Verde fuerte
    '#2980B9',  # Azul intenso
    '#C0392B',  # Rojo fuerte
    '#F39C12',  # Amarillo anaranjado
    '#BDC3C7'   # Gris suave
]


# Crear el gráfico de barras para cada columna
width = 0.25  # El ancho de cada barra

fig, ax = plt.subplots()

for i, col in enumerate(principales):
    ax.bar(x + i * width - (len(principales) - 1) * width / 2, df_principales[col], width, label=col[12::], color=colores[i])

# Añadir etiquetas y título
ax.set_xlabel('Regla', fontsize=14)
ax.set_ylabel('Clientes rechazados por la regla', fontsize=14)
ax.set_title('Principales reglas de rechazos en Precalificaciones Horus', fontsize=16)

# Posicionar la leyenda
ax.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=8)

# Ajustar las etiquetas del eje X
ax.set_xticks(x)
ax.set_xticklabels([f'Fila {i}' for i in range(len(df_principales))], fontsize=1)

# Mostrar el gráfico
plt.show()

In [ ]:
contagio_nivel_cliente["PATRONO"].value_counts().reset_index(name="Clientes").head(4)

plt.figure(figsize=(8,6))
dic_traduccion = {
    "Ministerio De Educacion": "MINEDUC",
    "Ministerio De Gobernacion": "MINGOB",
    "Ministerio Salud Publica Presupuesto": "MSPAS",
    "Estado (pasivas)": "Pasivas"
}

data = contagio_nivel_cliente["PATRONO"].value_counts().head(4)
data.index = data.index.map(lambda x: dic_traduccion.get(x, x))  
data.plot(kind="bar", color=["#dc0464", "#22aeae", 
                                                                   "#a3abb3", "#fbdc05",
                                                                   "#f2ebea"])


plt.title("Clientes según su patrono")
plt.xlabel("Patrono")
plt.ylabel("Clientes")
plt.show()

## Evaluar a quiénes sí vale la pena consolidar financieramente

In [ ]:
def vale_la_pena_consolidar(cat_interna: str, cat_externa: str, 
                            saldo_interno: float, saldo_externo: float) -> bool:
    dic_categorias = {"A": 0.036*0.45, "B": 0.45*0.086, "C": 0.45*0.156,
                      "D": 0.325*0.45, "E": 1}
    cat_interna = dic_categorias.get(cat_interna)
    cat_externa = dic_categorias.get(cat_externa)
    vale_la_pena = 0
    coeficiente_prueba = cat_interna * saldo_externo / saldo_interno + 2 * cat_interna
    try:
        if cat_externa >= coeficiente_prueba:
            vale_la_pena = 1
    except:
        vale_la_pena = 0
    return vale_la_pena

In [ ]:
conteo_nivel_cliente_temp = contagio_nivel_cliente.groupby(by="DPI")["saldo"].sum().reset_index()
conteo_nivel_cliente_temp.columns = ["DPI", "Saldo_externo"]

contagio_nivel_cliente = pd.merge(contagio_nivel_cliente, conteo_nivel_cliente_temp)
contagio_nivel_cliente["VALE LA PENA CONSOLIDAR"] = contagio_nivel_cliente.apply(lambda x: vale_la_pena_consolidar(
x["CATEGORIA_INTERNA"], x["PeorCategoria"], x["TOTAL_SALDO"], x["Saldo_externo"]
), axis=1)

## Evaluar oferta promedio

In [ ]:
# Reemplazar valores 1 y 0 por "Sí" y "No"
conteo = contagio_nivel_cliente["VALE LA PENA CONSOLIDAR"].replace({1: "Sí", 0: "No"}).value_counts()

# Graficar los datos
plt.figure(figsize=(8,6))
ax = conteo.plot(kind="bar", color=["#dc0464", "#22aeae"])

# Añadir etiquetas con la cantidad de clientes encima de cada barra
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() + p.get_width() / 2, p.get_height()), 
                ha='center', va='bottom')

# Etiquetas y título
plt.title("Clientes según si vale la pena consolidarlos")
plt.xlabel("¿Conviene financieramente?")
plt.ylabel("Clientes")
plt.show()

In [ ]:
# Reemplazar valores 1 y 0 por "Sí" y "No"
conteo = contagio_nivel_cliente["VALE LA PENA CONSOLIDAR"].replace({1: "Sí", 0: "No"}).value_counts()

# Graficar los datos en un gráfico de pastel
plt.figure(figsize=(8,6))
ax = conteo.plot(kind="pie", autopct='%1.1f%%', startangle=90, colors=["#22aeae", "#dc0464"])

# Título y configuración del gráfico
plt.title("Clientes según si vale la pena consolidarlos")
plt.ylabel("")  # Eliminar etiqueta del eje y, innecesaria en el gráfico de pastel
plt.show()

In [ ]:
def calcular_nuevo_rci(salario, tasa, plazo, saldo_interno, saldo_externo, rci_actual, cumple_15):
    nueva_oferta = calcular_cuota(saldo_externo, tasa, plazo) / salario
    if not cumple_15:
        nuevo =  nueva_oferta + rci_actual
    elif cumple_15:
        nuevo = nueva_oferta + calcular_cuota(saldo_interno, tasa, plazo) / salario
    return nuevo

# ahora hacer una estimacion con los que si valen la pena
mascara_consolidacion = contagio_nivel_cliente["VALE LA PENA CONSOLIDAR"] == 1

salario_promedio = contagio_nivel_cliente.loc[mascara_consolidacion]["DfIngresosValidados"].mean()
saldo_promedio_externo = contagio_nivel_cliente.loc[mascara_consolidacion]["Saldo_externo"].mean()
saldo_promedio_interno = contagio_nivel_cliente.loc[mascara_consolidacion]["TOTAL_SALDO"].mean()

estimador_tasa = 12


contagio_nivel_cliente["DfIngresosValidados"].fillna(salario_promedio, inplace=True)
contagio_nivel_cliente.loc[mascara_consolidacion, "NUEVO RCI INTERNO"] = contagio_nivel_cliente.loc[mascara_consolidacion].apply(lambda x: calcular_nuevo_rci(
    x["DfIngresosValidados"], estimador_tasa/1200, 120, x["TOTAL_SALDO"], x["Saldo_externo"], x["DfRciInternoInicialDeudor"], x["PAGA RESERVAS"]
), axis=1)



In [ ]:
plt.figure(figsize=(8,6))
plt.hist(contagio_nivel_cliente["NUEVO RCI INTERNO"], bins=700, color="#fbdc05")
plt.xlabel("Nuevo RCI Interno")
plt.xlim([0, 1.5])
plt.ylim([0,175])
plt.ylabel("Clientes")
plt.title(f"Histograma de Nuevo RCI Interno con simulación de TPP de {estimador_tasa}%")
plt.show()

In [ ]:
cant_clientes_viables = len(contagio_nivel_cliente[contagio_nivel_cliente["NUEVO RCI INTERNO"] < 0.6])
saldo_comprar = contagio_nivel_cliente[contagio_nivel_cliente["VALE LA PENA CONSOLIDAR"] == 1]["Saldo_externo"].sum()
promedio_saldo_cliente = contagio_nivel_cliente[contagio_nivel_cliente["VALE LA PENA CONSOLIDAR"] == 1]["Saldo_externo"].mean()
rci_promedio = contagio_nivel_cliente[(contagio_nivel_cliente["VALE LA PENA CONSOLIDAR"] == 1) & (contagio_nivel_cliente["NUEVO RCI INTERNO"] < 60)]["NUEVO RCI INTERNO"].mean()
ingreso_fin_promedio_12 = ingresos_financieros(promedio_saldo_cliente, estimador_tasa/1200, 120, 12)
reservas_evitadas = contagio_nivel_cliente[(contagio_nivel_cliente["NUEVO RCI INTERNO"] < 60) & (contagio_nivel_cliente["VALE LA PENA CONSOLIDAR"] == 1)]["RESERVAS A PAGAR CONTAGIO"].sum()

In [ ]:
print(f"Cantidad de clientes aplicables: {cant_clientes_viables}")
print(f"Saldo total a comprar {saldo_comprar / 10**6} M")
print(f"Promedio saldo a consolidar por cliente: {promedio_saldo_cliente / 10**3} K")
print(f"Nuevo RCI promedio: {rci_promedio}")
print(f"Ingreso financiero promedio a 12 meses por estos clientes y consoliacioens: {ingreso_fin_promedio_12}")
print(f"Reservas evitadas en estos clients: {reservas_evitadas / 10**6}")

In [ ]:
reservas_inevitables = contagio_nivel_cliente[(contagio_nivel_cliente["VALE LA PENA CONSOLIDAR"] == 0) | (contagio_nivel_cliente["NUEVO RCI INTERNO"] >= 60)]["RESERVAS A PAGAR CONTAGIO"].sum()
saldo_que_contagiar = contagio_nivel_cliente[(contagio_nivel_cliente["VALE LA PENA CONSOLIDAR"] == 0) | (contagio_nivel_cliente["NUEVO RCI INTERNO"] >= 60)]["TOTAL_SALDO"].sum()

print(f"Reservas inevitables: {reservas_inevitables / 10**6} M")
print(f"Saldo que inevitablemente se va a contagiar: {saldo_que_contagiar / 10**6} M")

In [ ]:
len(contagio_nivel_cliente)

## Estudio 2025

In [ ]:
base_contagio_nivel_cliente = base_contagio.drop_duplicates(subset="DPI")
cat_a_ext = len(base_contagio_nivel_cliente[base_contagio_nivel_cliente["PeorCategoria"] == "A"]) / len(base_contagio_nivel_cliente)
cat_b_ext = len(base_contagio_nivel_cliente[base_contagio_nivel_cliente["PeorCategoria"] == "B"]) / len(base_contagio_nivel_cliente)

cat_a_int = len(base_contagio_nivel_cliente[base_contagio_nivel_cliente["CATEGORIA_INTERNA"] == "A"]) / len(base_contagio_nivel_cliente)
cat_b_int = len(base_contagio_nivel_cliente[base_contagio_nivel_cliente["CATEGORIA_INTERNA"] == "B"]) / len(base_contagio_nivel_cliente)

potencial_reservas_ext = (0.036*0.45 * cat_a_ext + 0.45*0.086* cat_b_ext) * base_contagio_nivel_cliente["SALDO EXTERNO CLIENTE"].sum()
potencial_reservas_int = (0.036*0.45 * cat_b_int + 0.45*0.086 * cat_b_int) * base_contagio_nivel_cliente["TOTAL_SALDO"].sum()

reservas_2025 = potencial_reservas_ext 
reservas_2025 / 10**6

In [ ]:
## Ahora veamos los que no tienen categoría
cat_c_interna = len(base_contagio_nivel_cliente[base_contagio_nivel_cliente["PeorCategoria"] == "C"]) / len(base_contagio_nivel_cliente)
cat_d_interna = len(base_contagio_nivel_cliente[base_contagio_nivel_cliente["PeorCategoria"] == "D"]) / len(base_contagio_nivel_cliente)
cat_e_interna = len(base_contagio_nivel_cliente[base_contagio_nivel_cliente["PeorCategoria"] == "E"]) / len(base_contagio_nivel_cliente)

reservas = np.array([0.036*0.45, 0.45*0.086, 0.45*0.156, 0.45*0.325, 1])
proporciones = np.array([cat_a_int, cat_b_int, cat_c_interna, cat_d_interna, cat_e_interna])
#reservas_sin_cat = np.dot(reservas, proporciones) * base_contagio_nivel_cliente.loc[base_contagio_nivel_cliente["CATEGORIA_INTERNA"] == ]["TOTAL_SALDO"].sum()
#reservas_sin_cat / 10**6

In [ ]:
proporciones_categoria = base_contagio_nivel_cliente["PeorCategoria"].value_counts().reset_index(name="Clientes")
proporciones_categoria["Proporción"] = proporciones_categoria["Clientes"] / 104814
proporciones_categoria.rename(columns={"PeorCategoria": "Categoría", "Proporción": "Probabilidad de transición"}, inplace=True)
proporciones_categoria["Probabilidad de transición"] = proporciones_categoria["Probabilidad de transición"] * 100
proporciones_categoria[["Categoría", "Probabilidad de transición"]].to_latex("../db/Probabilidades de transición.txt")

In [ ]:
proporciones_categoria

In [ ]:
base_contagio["FechaSaldos"].unique()

## Sacar reservas 2025: ARIMA

In [163]:
from math import log
historial = pd.read_excel("../db/historial_reservas.xlsx")
historial["Reserva contagio"] = pd.to_numeric(historial["Reserva contagio"])
historial["Reserva contagio"] = historial.apply(lambda x: log(x["Reserva contagio"]), axis=1)
historial

,Fecha,Saldo total,Reserva contagio,PCT
0,2024-06-30,2.481152e+10,17.891043,0.237316
1,2024-07-31,2.500737e+10,17.840563,0.223867
2,2024-09-30,2.532904e+10,18.047357,0.271800
3,2024-08-31,2.518987e+10,18.013100,0.264097
4,2024-10-31,2.555899e+10,18.074393,0.276736


In [164]:
## Revisar premisas para aplicar sarima o arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.stattools import adfuller  
from statsmodels.tsa.statespace.sarimax import SARIMAX

def check_stationarity(timeseries):
    result = adfuller(timeseries.dropna())
    return result[1]

def make_stationary(df, column):
    p_value = check_stationarity(df[column])
    d = 0
    while p_value > 0.05:
        df[column] = df[column].diff().dropna()
        p_value = check_stationarity(df[column])
        d += 1
        print(f'{d}-esima diferneciación: p-valor = {p_value}')
    return df, d

In [165]:
for col in historial.columns.tolist():
    try:
        historicos_estacionaria, d = make_stationary(historial.copy(), col)
    except:
        pass

1-esima diferneciación: p-valor = 0.49494230539481526
1-esima diferneciación: p-valor = 0.11001276896927009
1-esima diferneciación: p-valor = 0.004864157329717941
1-esima diferneciación: p-valor = 0.0050259013016902415


In [166]:
import itertools

best_aic_indicator = float("inf")
best_pdq_indicator = None
best_seasonal_pdq_indicator = None

p = q = range(0, 3)
d = range(0,1)
pdq = list(itertools.product(p, d, q))

# seasonal parameter
seasonal_pdq = [(x[0], x[1], x[2], 3) for x in pdq]

parametros = {}
# guardar en nomnre col, parámetro y parámetro estacionario
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = SARIMAX(historial["Reserva contagio"], order=param,
                        seasonal_order=param_seasonal,
                        enforce_stationarity=False,
                        enforce_invertibility=False)
            results = mod.fit(disp=False)
            if results.aic < best_aic_indicator:
                best_aic_indicator = results.aic
                best_pdq_indicator = param
                best_seasonal_pdq_indicator = param_seasonal
        except Exception as e:
            print(f"Error with parameters {param} & {param_seasonal}: {e}")
            continue
parametros["Reserva contagio"] = (param, param_seasonal)

c:\Users\mariajose_chinchilla\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\mariajose_chinchilla\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\mariajose_chinchilla\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\mariajose_chinchilla\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge.

In [ ]:
predicciones_df = pd.DataFrame(index=pd.date_range(start='2025-01-01', periods=12, freq='M'))

modelo = SARIMAX(historicos_estacionaria["Reserva contagio"],
                        order=(1,1,1),
                        seasonal_order=(2,2,1,2),
                        enforce_invertibility=False,
                        enforce_stationarity=False)

modelo_fitted = modelo.fit(disp=False)
pred = modelo_fitted.forecast(steps=12)

# Crear índice de fechas desde enero 2025 a diciembre 2025 para las predicciones
pred.index = pd.date_range(start='2025-01-01', periods=12, freq='M')

# Almacenar las predicciones en el DataFrame
predicciones_df["Predicciones de reserva"] = pred

from math import exp
predicciones_df["Predicciones de reserva"] = predicciones_df.apply(lambda x: exp(x["Predicciones de reserva"]), axis=1)
predicciones_df

,Predicciones de reserva
2025-01-31,9.983704e+07
2025-02-28,2.423826e+05
2025-03-31,2.881793e+05
2025-04-30,4.668940e+05
2025-05-31,2.847837e+00
2025-06-30,3.786332e+00
2025-07-31,7.041209e+00
2025-08-31,1.079024e-07
2025-09-30,1.604268e-07
2025-10-31,3.424354e-07


In [185]:
predicciones_df["Predicciones de reserva"].max() * 1.02 * 2 / 10**6

203.66756276960285